In [ ]:
import networkx as nx
import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt


In [ ]:
timus_G = nx.read_edgelist('./nx_create/nx_graph_df_timus.csv'
                     , create_using=nx.DiGraph()
                     , nodetype=int
                     , data=[('weight', int)]
                     )


In [ ]:
list(timus_G.edges(data=True))


In [ ]:
len(list(timus_G.edges(data=True)))

In [ ]:
pos = nx.spring_layout(timus_G) 
plt.figure(figsize = (15, 15)) 
node_item_id = 1001
#networkx.draw_networkx_labels(purchasedAsinEgoGraph, pos, font_size = 18) 
edgewidth = [d['weight'] for (u, v, d) in timus_G.edges(data = True)]
nx.draw(timus_G, pos = pos, node_size = 50, node_color='g', edge_color= edgewidth, style = 'solid')
nx.draw_networkx_nodes(timus_G, pos = pos, nodelist= [node_item_id], node_color= 'r', node_size = 50)
plt.show()

In [ ]:
# std = 14
threshold = 3
GivenNodeIslands = nx.Graph()
for f, t, e in timus_G.edges(data = True):
    if e['weight'] >= threshold:
        GivenNodeIslands.add_edge(f, t, weight = e['weight'])
node_TrimGraph = nx.Graph(GivenNodeIslands)

In [ ]:
plt.figure(figsize = (10, 10)) 
neg1 = 1000
# (1000, 1001, {'weight': 453})
neg2 = 2056
#(1000, 2056, {'weight': 14})
#networkx.draw_networkx_labels(purchasedAsinEgoTrimGraph, pos, font_size = 18) 
edgewidth = [d['weight'] for (u, v, d) in node_TrimGraph.edges(data = True)]
nx.draw_networkx_nodes(node_TrimGraph, pos = pos, nodelist= nx.nodes(node_TrimGraph), node_color='g') 
nx.draw_networkx_edges(node_TrimGraph, pos = pos, edgelist= node_TrimGraph.edges)
nx.draw_networkx_nodes(node_TrimGraph, pos = pos, nodelist= [node_item_id], node_color= 'r') #ego node
nx.draw_networkx_nodes(node_TrimGraph, pos = pos, nodelist= [neg1], node_color= 'b') #ego node
nx.draw_networkx_nodes(node_TrimGraph, pos = pos, nodelist= [neg2], node_color= 'r') #ego node
plt.show()
#共现次数越大越近


In [ ]:
# demo[1001],demo[1000],demo[2056]

In [ ]:
node_TrimGraph.edges(data = True)

In [ ]:
nx.number_of_nodes(node_TrimGraph)

In [ ]:

# pr = nx.pagerank(timus_G)
# pr

In [ ]:
timus = pd.read_csv('./nx_create/nx_graph_df_timus.csv',header=None,sep=' ',names=['source','target','weight'])

In [ ]:
timus

In [ ]:
from tqdm import tqdm
node_dict_degree_clus = {}
pr = nx.pagerank(timus_G)

degree_centralityG = nx.degree_centrality(timus_G)

eigenvector_centralityG = nx.eigenvector_centrality(timus_G,weight='weight')

average_degree_connectivityG = nx.average_degree_connectivity(timus_G, weight="weight")

harmonic_centralityG = nx.harmonic_centrality(timus_G,distance='weight')

betweenness_centralityG = nx.betweenness_centrality(timus_G,weight='weight')

for node_id in tqdm(nx.nodes(timus_G)):
    metadata = {}
    metadata['PageRank'] = round(pr[node_id],5)
    metadata['DegreeCentrality'] = round(degree_centralityG[node_id],5)
    metadata['EigenvectorCentrality'] = round(eigenvector_centralityG[node_id],5)
    metadata['HarmonicCentrality'] = round(harmonic_centralityG[node_id],5)
    metadata['BetweennessCentrality'] = round(betweenness_centralityG[node_id],5)
    ego = nx.ego_graph(timus_G, node_id, distance='weight')
    metadata['ClusteringCoeff'] = round(nx.average_clustering(ego,weight='weight'),5)
    node_dict_degree_clus[node_id] = metadata

   

In [ ]:
node_dict_degree_clus

In [ ]:
import pickle
pickle.dump(node_dict_degree_clus, open( 'timus_node_dict_Centrality_clus.pkl', 'wb'))   

In [ ]:
# demo = pd.DataFrame(node_dict_degree_clus)

In [ ]:
# demo

In [ ]:
node_dict_degree_clus.items()

In [ ]:

timus_sna = []
for item_id,metadata in tqdm(node_dict_degree_clus.items()):
    singledata = {}
    singledata['item_id'] = item_id
    singledata['PageRank'] = metadata['PageRank']
    singledata['DegreeCentrality'] = metadata['DegreeCentrality']
    singledata['EigenvectorCentrality'] = metadata['EigenvectorCentrality']
    singledata['HarmonicCentrality'] = metadata['HarmonicCentrality']
    singledata['BetweennessCentrality'] = metadata['BetweennessCentrality']
    singledata['ClusteringCoeff'] = metadata['ClusteringCoeff']
    timus_sna.append(singledata)
    

In [ ]:
timus_sna_analysis = pd.DataFrame(timus_sna)


In [ ]:
timus_sna_analysis_item_id = timus_sna_analysis.set_index('item_id')


In [ ]:
timus_sna_analysis_item_id

In [ ]:
node_item_id_Neighbors = [n for n in node_TrimGraph.neighbors(node_item_id)]
node_item_id_Neighbors

In [ ]:
sanple_item_id_Neighbors = [n for n in node_TrimGraph.neighbors(node_item_id)]
sanple_item_id_Neighbors

In [ ]:
# timus_sna_analysis_item_id.loc[sanple_item_id_Neighbors]

In [ ]:
import seaborn as sns
timus_sna_analysis_item_id.corr()

In [ ]:
import numpy as np
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

timus_sna_analysis['HarmonicCentrality'] = timus_sna_analysis[['HarmonicCentrality']].apply(max_min_scaler)
timus_sna_analysis_item_id['HarmonicCentrality'] = timus_sna_analysis_item_id[['HarmonicCentrality']].apply(max_min_scaler)


##
timus_sna_analysis['PageRank'] = timus_sna_analysis[['PageRank']].apply(max_min_scaler)
timus_sna_analysis_item_id['PageRank'] = timus_sna_analysis_item_id[['PageRank']].apply(max_min_scaler)

##
timus_sna_analysis['DegreeCentrality'] = timus_sna_analysis[['DegreeCentrality']].apply(max_min_scaler)
timus_sna_analysis_item_id['DegreeCentrality'] = timus_sna_analysis_item_id[['DegreeCentrality']].apply(max_min_scaler)
##

timus_sna_analysis['EigenvectorCentrality'] = timus_sna_analysis[['EigenvectorCentrality']].apply(max_min_scaler)
timus_sna_analysis_item_id['EigenvectorCentrality'] = timus_sna_analysis_item_id[['EigenvectorCentrality']].apply(max_min_scaler)
#
timus_sna_analysis['ClusteringCoeff'] = timus_sna_analysis[['ClusteringCoeff']].apply(max_min_scaler)
timus_sna_analysis_item_id['ClusteringCoeff'] = timus_sna_analysis_item_id[['ClusteringCoeff']].apply(max_min_scaler)
#
# BetweennessCentrality
timus_sna_analysis['BetweennessCentrality'] = timus_sna_analysis[['BetweennessCentrality']].apply(max_min_scaler)
timus_sna_analysis_item_id['BetweennessCentrality'] = timus_sna_analysis_item_id[['BetweennessCentrality']].apply(max_min_scaler)
##

In [ ]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)

In [ ]:
neigh.fit(timus_sna_analysis[['PageRank','DegreeCentrality','EigenvectorCentrality','HarmonicCentrality','BetweennessCentrality','ClusteringCoeff']].values)

In [ ]:
# demo1.loc[1001,:].values.reshape((1,-1))

In [ ]:
ind = neigh.kneighbors(timus_sna_analysis_item_id.loc[1409,:].values.reshape((1,-1)),10, return_distance=False)

In [ ]:
timus_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()

In [ ]:
recall_item_num = 11
timus = pd.read_csv('./nx_create/nx_graph_df_timus.csv',header=None,sep=' ',names=['source','target','weight'])
# timus = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
val_source_target = []
for _ ,group in timus.groupby('source'):
#     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
    tmp = group.iloc[0,:]
    val_source_target.append((tmp.source,tmp.target))
val_source_target
source_target = list(zip(timus.source.tolist(),timus.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(timus_sna_analysis['item_id'].unique()):
    ind = neigh.kneighbors(timus_sna_analysis_item_id.loc[iid,:].values.reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = timus_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[1:]
#     print(item_recall_list)

# eva_data = []
# for iid,group in timus.groupby('source'):
#     if len(group)>10:
#         tmp = list(zip(group.source,group.target))[:10]
#     else: tmp = list(zip(group.source,group.target))
#     eva_data.extend(tmp[:])   


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
#         for rank,item in enumerate(tmp_recall_items):
#             for s,t in eva_data:
#                 if s==iid and item==t:
# #                     print("标注：",s,t)
#                     score = 1.0/(rank+1.0)

#                     print("标注",s,t)
#                     break
            
#     for iid,recall_list in item_recall_list:
#         tmp_recall_items = item_recall_list[iid][:k]
#         for rank,item in enumerate(tmp_recall_items):
#             for s,t in source_target:
#                 if s==iid and item==t:
#                     score += 1.0/(rank+1.0)
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        


In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
item_recall_list

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
###scaler
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
plt.subplots(1,1,figsize=(36,36))
nx.draw(timus_G,with_labels=True, font_weight='bold',width = 0.5)
# nx.draw(timus_G, font_weight='bold',width = 0.5)
plt.axis('on')
plt.show()

In [ ]:
neigh1 = NearestNeighbors(n_neighbors=1)
neigh1.fit(timus_sna_analysis[['DegreeCentrality','EigenvectorCentrality','HarmonicCentrality','BetweennessCentrality']].values)

In [ ]:
recall_item_num = 11
timus = pd.read_csv('./nx_create/nx_graph_df_timus.csv',header=None,sep=' ',names=['source','target','weight'])
# timus = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
val_source_target = []
for _ ,group in timus.groupby('source'):
#     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
    tmp = group.iloc[0,:]
    val_source_target.append((tmp.source,tmp.target))
val_source_target
source_target = list(zip(timus.source.tolist(),timus.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(timus_sna_analysis['item_id'].unique()):
    ind = neigh1.kneighbors(timus_sna_analysis_item_id.loc[iid,['DegreeCentrality','EigenvectorCentrality','HarmonicCentrality','BetweennessCentrality']].values.reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = timus_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[1:]
#     print(item_recall_list)

# eva_data = []
# for iid,group in timus.groupby('source'):
#     if len(group)>10:
#         tmp = list(zip(group.source,group.target))[:10]
#     else: tmp = list(zip(group.source,group.target))
#     eva_data.extend(tmp[:])   


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
#         for rank,item in enumerate(tmp_recall_items):
#             for s,t in eva_data:
#                 if s==iid and item==t:
# #                     print("标注：",s,t)
#                     score = 1.0/(rank+1.0)

#                     print("标注",s,t)
#                     break
            
#     for iid,recall_list in item_recall_list:
#         tmp_recall_items = item_recall_list[iid][:k]
#         for rank,item in enumerate(tmp_recall_items):
#             for s,t in source_target:
#                 if s==iid and item==t:
#                     score += 1.0/(rank+1.0)
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        


In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)